In [2]:
import sys
import torch
import mlflow
import numpy as np
import pandas as pd
from datetime import datetime
from darts import TimeSeries, concatenate
import plotly.express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt
from darts.metrics import mae, mse, mape, rmse, smape
from darts.dataprocessing.transformers.scaler import Scaler
from pytorch_lightning.callbacks import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback
from darts.models import RNNModel, BlockRNNModel, TCNModel, TransformerModel, TCNModel, NBEATSModel

sys.path.append('../utils/')

from config import config

freq='2H3T14S'
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment("OpenMarsHyperOptFixedHorizon")
pd.options.plotting.backend = "plotly"


In [3]:
def load_dataset(training_file, testing_file):
    dataframes = []
    for data_file in [training_file, testing_file]:
        parser = lambda data_string: datetime.strptime(data_string, '%Y-%m-%d %H:%M:%S')
        dataframe = pd.read_csv(data_file, parse_dates=['Time'],
                                date_parser=parser)
        print(f"Rows in {data_file}: {len(dataframe)}")
        dataframe.drop(['Ls', 'LT', 'CO2ice'], axis=1, inplace=True)
        dataframes.append(dataframe)

    return pd.concat(dataframes, axis=0)


def preprocess(dataframe):
        time = pd.date_range("1998-07-15 21:23:39", periods=len(dataframe), freq=freq)
        dataframe.index = time
        dataframe = dataframe.drop(['Time'], axis=1)
        return dataframe

def create_series(dataframe):
        series = TimeSeries.from_dataframe(dataframe, time_col=None, value_cols=None, fill_missing_dates=True, freq=freq, fillna_value=None)
        return series.astype(np.float32)

def create_train_val_test_series(series):
        train, temp = series.split_after(0.7)
        val, test = temp.split_after(0.67)
        return train, val, test

In [4]:
dataframe = load_dataset('../data/data_files/insight_openmars_training_time.csv',
                         '../data/data_files/insight_openmars_test_time.csv')
dataframe = preprocess(dataframe)
full_series = create_series(dataframe)
train, val, test = create_train_val_test_series(full_series)
# print(len(train), len(val), len(test))
len(full_series)

/tmp/ipykernel_27399/3478556548.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  dataframe = pd.read_csv(data_file, parse_dates=['Time'],


Rows in ../data/data_files/insight_openmars_training_time.csv: 72196
Rows in ../data/data_files/insight_openmars_test_time.csv: 16364


/tmp/ipykernel_27399/3478556548.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  dataframe = pd.read_csv(data_file, parse_dates=['Time'],


88560

In [5]:
val.components

Index(['Tsurf', 'Psurf', 'cloud', 'vapour', 'u_wind', 'v_wind', 'dust',
       'temp'],
      dtype='object', name='component')

In [6]:
scaler = Scaler()  # default uses sklearn's MinMaxScaler
full_series = scaler.fit_transform(full_series)
train = scaler.fit_transform(train)
val = scaler.transform(val)
test = scaler.transform(test)

lr_scheduler_cls = torch.optim.lr_scheduler.ExponentialLR
models = {}

In [7]:
print(len(train), len(val), len(test))

61992 17800 8768


In [11]:
def create_block_rnn_model():

    # select input and output chunk lengths
    in_len = 84
    out_len =  12
    batch_size = 96

    # Other hyperparameters
    n_rnn_layers = 2
    hidden_dim =  30
    dropout = 0.25
    lr = 0.0005
    # reproducibility
    torch.manual_seed(42)

    # pruner = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.0008, patience=3, verbose=False)
    callbacks = [early_stopper]

    pl_trainer_kwargs = {
    "gradient_clip_val": 1,
    "max_epochs": 50,
    "accelerator": "auto",
    "callbacks": callbacks,
    }

    common_model_args = {
        "optimizer_kwargs": {'lr': lr},
        "pl_trainer_kwargs": pl_trainer_kwargs,
        "lr_scheduler_cls": lr_scheduler_cls,
        "lr_scheduler_kwargs": config.lr_scheduler_kwargs,
        "likelihood": None,  # use a likelihood for probabilistic forecasts
        "save_checkpoints": True,  # checkpoint to retrieve the best performing model state,
        "force_reset": True,
        "random_state": 42,

    }
    # build the BlockRNNModel model
    model = BlockRNNModel(model = "LSTM",
                                input_chunk_length= in_len,
                                output_chunk_length = out_len,
                                n_rnn_layers = n_rnn_layers,
                                hidden_dim = hidden_dim,
                                batch_size = batch_size,
                                model_name="BlockRNNModel_84_12", 
                                dropout = dropout,
                                **common_model_args)
    return model


def create_tcn_model():
    in_len = 84
    out_len =  12
    batch_size = 96
    # Other hyperparameters
    kernel_size = 2
    num_filters = 6
    weight_norm = True
    dilation_base = 2
    dropout = 0.05
    lr = 0.0005
    # reproducibility
    torch.manual_seed(42)


    # pruner = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.0008, patience=3, verbose=False)
    callbacks = [early_stopper]



    pl_trainer_kwargs = {
    "gradient_clip_val": 1,
    "max_epochs": 50,
    "accelerator": "auto",
    "callbacks": callbacks,
    }

    common_model_args = {
        "optimizer_kwargs": {'lr': lr},
        "pl_trainer_kwargs": pl_trainer_kwargs,
        "lr_scheduler_cls": lr_scheduler_cls,
        "lr_scheduler_kwargs": config.lr_scheduler_kwargs,
        "likelihood": None,  # use a likelihood for probabilistic forecasts
        "save_checkpoints": True,  # checkpoint to retrieve the best performing model state,
        "force_reset": True,
        "batch_size": batch_size,
        "random_state": 42,

    }

    # build the TCN model
    model = TCNModel(
    input_chunk_length= in_len,
    output_chunk_length = out_len,
    dilation_base = dilation_base,
    weight_norm = weight_norm,
    kernel_size = kernel_size,
    num_filters = num_filters,
    model_name = 'TCNModel_84_12',
    dropout = dropout,
    **common_model_args
    )

    return model

def create_transformer_model():
    in_len = 84
    out_len =  12
    batch_size = 96

    # Other hyperparameters

    d_model=12
    nhead=6

    num_encoder_layers=2
    num_decoder_layers=4
    dim_feedforward=64
    dropout = 0.05
    lr = 0.0005
    torch.manual_seed(42)


    early_stopper = EarlyStopping("val_loss", min_delta=0.0008, patience=3, verbose=False)
    callbacks = [early_stopper]



    pl_trainer_kwargs = {
    "gradient_clip_val": 1,
    "max_epochs": 50,
    "accelerator": "auto",
    "callbacks": callbacks,
    }

    common_model_args = {
        "optimizer_kwargs": {'lr': lr},
        "pl_trainer_kwargs": pl_trainer_kwargs,
        "lr_scheduler_cls": lr_scheduler_cls,
        "lr_scheduler_kwargs": config.lr_scheduler_kwargs,
        "likelihood": None,  # use a likelihood for probabilistic forecasts
        "save_checkpoints": True,  # checkpoint to retrieve the best performing model state,
        "force_reset": True,
        "batch_size": batch_size,
        "random_state": 42,

    }

    # build the Transfofrmer model
    model = TransformerModel(
    input_chunk_length=in_len,
    output_chunk_length=out_len,
    d_model=d_model,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    dim_feedforward=dim_feedforward,
    dropout=dropout,
    activation="relu",
    model_name = 'TransformerModel_84_12',
    **common_model_args,
)
    
    return model


def create_n_beats_model():
    in_len = 84
    out_len =  12
    batch_size = 96
    lr = 0.0005
    torch.manual_seed(42)

    # Other hyperparameters
    num_blocks=3
    num_layers=4
    layer_widths=512
    dropout = 0.05

    early_stopper = EarlyStopping("val_loss", min_delta=0.0008, patience=3, verbose=False)
    callbacks = [early_stopper]

    pl_trainer_kwargs = {
    "gradient_clip_val": 1,
    "max_epochs": 50,
    "accelerator": "auto",
    "callbacks": callbacks,
    }

    common_model_args = {
        "optimizer_kwargs": {'lr': lr},
        "pl_trainer_kwargs": pl_trainer_kwargs,
        "lr_scheduler_cls": lr_scheduler_cls,
        "lr_scheduler_kwargs": config.lr_scheduler_kwargs,
        "likelihood": None,  # use a likelihood for probabilistic forecasts
        "save_checkpoints": True,  # checkpoint to retrieve the best performing model state,
        "force_reset": True,
        "batch_size": batch_size,
        "random_state": 42,

    }

# build the NBeats model
    model = NBEATSModel(
        generic_architecture=False,
        num_blocks=num_blocks,
        num_layers=num_layers,
        layer_widths=layer_widths,
        loss_fn=torch.nn.MSELoss(),
        input_chunk_length= in_len,
        output_chunk_length = out_len,
        model_name = 'NBEATSModel_84_12',
        dropout=dropout,
        **common_model_args,

)
    return model

In [12]:

def forecast(model, predict_after_series = val, variable  = 'dust', forecast_type = 'backtest', n = 84, forecast_horizon = 12):
    result_accumulator = {}
    print(f'For model {model.model_name}')
    if forecast_type == 'backtest':
        pred_series = model.historical_forecasts(series=full_series, 
                                        past_covariates=None,
                                        future_covariates=None,
                                        start = test.start_time(),
                                        stride = forecast_horizon,
                                        retrain=False,
                                        verbose=False, 
                                        forecast_horizon=forecast_horizon,
                                        last_points_only = False
                                        )
        pred_series = concatenate(pred_series)

    else:
        pred_series = model.predict(series=predict_after_series, n = n)
    test_variable = test[variable]
    pred_variable = pred_series[variable]
    if variable != 'cloud':

        result_accumulator[variable] = {
            "mae": mae(test_variable, pred_variable),
            "mse": mse(test_variable, pred_variable),
            "mape": mape(test_variable, pred_variable),
            "rmse": rmse(test_variable, pred_variable)
        }
    else:
        result_accumulator[variable] = {
            "mae": mae(test_variable, pred_variable),
            "mse": mse(test_variable, pred_variable),
            "rmse": rmse(test_variable, pred_variable)
        }


    return result_accumulator, pred_series

In [13]:
def plotting(model, mlflow_run_id, test, pred_series, n = 84, type = 'backtest', variable = 'dust'):
      all_artifacts = []
      if type == 'backtest':
            df_to_plot = pd.DataFrame({'Actual': scaler.inverse_transform(test)[variable].pd_series(), model.model_name.split('_')[0]: scaler.inverse_transform(pred_series)[variable].pd_series()})
            fig = df_to_plot.plot(title=f"{variable} {type}", template="simple_white",
                  labels=dict(index="time", value=variable, variable="Legend"))
            fig.show()
            fig.write_image(f'../plots/{model.model_name}_{variable}_{type}.png')
            # plt.savefig(f'../plots/{model.model_name}_{variable}_{type}.png')
            all_artifacts.append(f'../plots/{model.model_name}_{variable}_{type}.png')

            df_to_plot_zoom = pd.DataFrame({'Actual': scaler.inverse_transform(test)[variable][pd.Timestamp('2018-06-05T20:07:47'):pd.Timestamp('2018-07-31T11:09:27')].pd_series(), model.model_name.split('_')[0]: scaler.inverse_transform(pred_series)[variable][pd.Timestamp('2018-06-05T20:07:47'):pd.Timestamp('2018-07-31T11:09:27')].pd_series()})
            fig2 = df_to_plot_zoom.plot(title=f"Zoomed {variable} {type}", template="simple_white",
                  labels=dict(index="time", value=variable, variable="Legend"))
            fig2.show()
            fig2.write_image(f'../plots/{model.model_name}_{variable}_{type}_zoomed.png')
            # plt.savefig(f'../plots/{model.model_name}_{variable}_{type}_zoomed.png')
            all_artifacts.append(f'../plots/{model.model_name}_{variable}_{type}_zoomed.png')
      elif type == 'predict':
            df_to_plot = pd.DataFrame({'Actual': scaler.inverse_transform(full_series[pd.Timestamp('2018-06-05T20:07:47'):])[variable].pd_series(), model.model_name.split('_')[0]: scaler.inverse_transform(pred_series)[variable].pd_series()})
            fig = df_to_plot.plot(title=f"{variable} {type}", template="simple_white",
                  labels=dict(index="time", value=variable, variable="Legend"))
            fig.show()
            fig.write_image(f'../plots/{model.model_name}_{variable}_{type}.png')
            # plt.savefig(f'../plots/{model.model_name}_{variable}_{type}.png')
            all_artifacts.append(f'../plots/{model.model_name}_{variable}_{type}.png')

            df_to_plot_zoom = pd.DataFrame({'Actual': scaler.inverse_transform(full_series)[variable][pd.Timestamp('2018-06-05T20:07:47'):pd.Timestamp('2018-07-31T11:09:27')].pd_series(), 
                                                model.model_name.split('_')[0]: scaler.inverse_transform(pred_series)[variable].pd_series()})
            fig2 = df_to_plot_zoom.plot(title=f"Zoomed {variable} {type}", template="simple_white",
                  labels=dict(index="time", value=variable, variable="Legend"))
            fig2.show()
            fig2.write_image(f'../plots/{model.model_name}_{variable}_{type}_zoomed.png')
            # plt.savefig(f'../plots/{model.model_name}_{variable}_{type}_zoomed.png')
            all_artifacts.append(f'../plots/{model.model_name}_{variable}_{type}_zoomed.png')

      return all_artifacts


In [14]:
# all_models =    {'BlockRNNModel_84_12': create_block_rnn_model(),
#                 'TCNModel_84_12': create_tcn_model(),
#                 'TransformerModel_84_12': create_transformer_model(),
#                 'NBEATSModel_84_12': create_n_beats_model(),
#                   }

# all_models =    {'BlockRNNModel_84_12': BlockRNNModel.load_from_checkpoint(model_name = "BlockRNNModel_84_12", best=True),
#                 'TCNModel_84_12': TCNModel.load_from_checkpoint(model_name ="TCNModel_84_12", best=True),
#                 'TransformerModel_84_12': TransformerModel.load_from_checkpoint(model_name ="TransformerModel_84_12", best=True),
#                 'NBEATSModel_84_12': NBEATSModel.load_from_checkpoint(model_name ="NBEATSModel_84_12", best=True),
#                   }

# for model_name, model in all_models.items():
#         # detect if a GPU is available
#     if torch.cuda.is_available():
#         num_workers = 8
#     else:
#         num_workers = 0

#     series = create_series(dataframe)

#     # when validating during training, we can use a slightly longer validation
#     # set which also contains the first input_chunk_length time steps
#     model_val_set = val

#     # train the model
#     model.fit(
#         series=train,
#         val_series=model_val_set,
#         num_loader_workers=num_workers,
    # )


In [1]:
trained_models =    {'BlockRNNModel_84_12_HyperOpt': BlockRNNModel.load_from_checkpoint(model_name = "BlockRNNModel_84_12_HyperOpt", best=True),
                'TCNModel_84_12_HyperOpt': TCNModel.load_from_checkpoint(model_name ="TCNModel_84_12_HyperOpt", best=True),
                'TransformerModel_84_12_HyperOpt': TransformerModel.load_from_checkpoint(model_name ="TransformerModel_84_12_HyperOpt", best=True),
                'NBEATSModel_84_12_HyperOpt_HyperOpt': NBEATSModel.load_from_checkpoint(model_name ="NBEATSModel_84_12_HyperOpt", best=True),
                  }
# trained_models =    {
#                 'NBEATSModel_84_12_HyperOpt': NBEATSModel.load_from_checkpoint(model_name ="NBEATSModel_84_12_HyperOpt", best=True),
#                   }


all_variables = ['dust', 'Tsurf', 'Psurf', 'cloud', 'vapour', 'u_wind', 'v_wind', 'temp']
all_variables = ['dust']

types = ['backtest', 'predict']
for model_name, model in trained_models.items():
    with mlflow.start_run(run_name=model.model_name) as run:
        all_artifacts = []
        mlflow.set_tag("Model_Name", model.model_name)
        mlflow.log_params(model.model_params)
        for variable in all_variables:
            
            for type in types:
                metrics = {}
                if type == 'predict':
                    series = full_series[:pd.Timestamp('2018-06-05T20:07:47')]
                    eval_results, pred_series = forecast(model, predict_after_series=series,
                                                          variable=variable, forecast_type=type)
                    metrics[type] = eval_results
                    # mlflow.log_artifact(metrics)
                    
                else:
                    series = test
                    eval_results, pred_series = forecast(model, predict_after_series=series,
                                                          variable=variable, forecast_type=type)
                    metrics[type] = eval_results
                    # mlflow.log_artifact(metrics)
                print(eval_results)
                if type == 'backtest':
                    for var, metric_dict in eval_results.items():
                        mlflow.log_metrics(metric_dict)
                        # for metric_name, val in metric_dict.items():
                        #     mlflow.log_metrics(metric)
                mlflow.log_table(data = eval_results, artifact_file = f'metrics_{model.model_name}_{variable}_{type}.json')
                artifacts = plotting(model, run.info.run_id, test, pred_series, type = type,  variable = variable)
                all_artifacts.extend(artifacts)
                columns = ['Tsurf', 'Psurf', 'cloud', 'vapour', 'u_wind', 'v_wind', 'dust', 'temp']
                predict_df = scaler.inverse_transform(pred_series).pd_dataframe()
                predicted_file_nm =f'{model.model_name}_{type}'
                final_path = f'/home/ubuntu/OpenMarsML/data/predicted_data/{predicted_file_nm}.csv'
                print(f'Going to save predictions of model {model.model_name} to file {final_path}')
                predict_df.to_csv(final_path)

        for artifact in all_artifacts:
            print(artifact)
            mlflow.log_artifact(artifact)


NameError: name 'BlockRNNModel' is not defined

In [13]:
# series_to_predict = full_series[:pd.Timestamp('2018-06-05T20:07:47')]
# series_predicted_actual = full_series[:pd.Timestamp('2018-06-05 22:11:01')]
# final_path = f'/home/ubuntu/OpenMarsML/data/data_files/series_to_predict.csv'
# series_to_predict.to_csv(f'/home/ubuntu/OpenMarsML/data/data_files/series_to_predict.csv')
# series_predicted_actual.to_csv(f'/home/ubuntu/OpenMarsML/data/data_files/predicted_series_actual_vals.csv')
